# XGBoost (No Rolling Window)

## Preprocessing

- Implement Log returns calculation
- Normalize or scale key features
- Encode log return to binary value (one-hot encoding)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, f1_score, 
                            confusion_matrix, roc_auc_score,
                            RocCurveDisplay)

df = pd.read_csv("./filtered_stocks_combined.csv")

# Change Date to DateTime
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(['ticker', 'Date'])

# Create target variable
# 1 if current close > prev close, else 0
df['prev_close'] = df.groupby('ticker')['Close'].shift(1)
df['target'] = (df['Close'] > df['prev_close']).astype(int)
df = df.dropna(subset=['prev_close'])

FileNotFoundError: [Errno 2] No such file or directory: '../filtered_stocks_combined.csv'

In [ ]:
df.head()

## Split df into training/test set

In [ ]:
features = ['Open', 'High', 'Low', 'Volume']
X = df[features]
y = df['target']
groups = df['ticker']

# Split off 80:20 for train:test
splitter = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, test_idx = next(splitter.split(X, y, groups=df['ticker']))

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Hyperparameter Tuning

In [ ]:
tuning_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    tree_method='hist',     
    booster='gbtree',     
    device='cuda',    
    random_state=42,                      
    eval_metric='auc',
    n_jobs=1              
)

### RandomizedSearchCV

In [ ]:
rs_params = {
    "learning_rate":    [0.01, 0.03, 0.05, 0.1, 0.2, 0.3],
    "max_depth":        [3, 4, 5, 6, 8, 10, 12],
    "min_child_weight": [1, 2, 3, 5, 7, 10],
    "gamma":            [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    "subsample":        [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "reg_alpha":        [0, 0.001, 0.01, 0.1, 1],
    "reg_lambda":       [0.5, 1, 1.5, 2, 5],
}

random_search = RandomizedSearchCV(
    tuning_clf,
    param_distributions=rs_params,
    n_iter=20,
    scoring='roc_auc',
    n_jobs=1,
    cv=5,
    verbose=1
)

random_search.fit(X_train_scaled, y_train)

In [ ]:
random_search.best_estimator_
random_search.best_params_

### GridSearchCV

In [ ]:
best_rs_params = random_search.best_params_

gs_params = {
    "learning_rate": [
        max(0.01, best_rs_params["learning_rate"] * 0.5),
        best_rs_params["learning_rate"],
        min(0.3,  best_rs_params["learning_rate"] * 1.5)
    ],

    "max_depth": sorted(list(set([
        max(1, best_rs_params["max_depth"] - 1),
        best_rs_params["max_depth"],
        best_rs_params["max_depth"] + 1
    ]))),

    "min_child_weight": sorted(list(set([
        max(1, best_rs_params["min_child_weight"] - 1),
        best_rs_params["min_child_weight"],
        best_rs_params["min_child_weight"] + 1
    ]))),

    "gamma": [
        max(0.0, best_rs_params["gamma"] - 0.1),
        best_rs_params["gamma"],
        best_rs_params["gamma"] + 0.1
    ],

    "subsample": [
        max(0.5, best_rs_params["subsample"] - 0.1),
        best_rs_params["subsample"],
        min(1.0, best_rs_params["subsample"] + 0.1)
    ],
    "colsample_bytree": [
        max(0.5, best_rs_params["colsample_bytree"] - 0.1),
        best_rs_params["colsample_bytree"],
        min(1.0, best_rs_params["colsample_bytree"] + 0.1)
    ],

    "reg_alpha": [
        max(0.0, best_rs_params["reg_alpha"] / 2),
        best_rs_params["reg_alpha"],
        best_rs_params["reg_alpha"] * 2
    ],
    "reg_lambda": [
        max(0.1, best_rs_params["reg_lambda"] / 2),
        best_rs_params["reg_lambda"],
        best_rs_params["reg_lambda"] * 2
    ]
}

grid_search = GridSearchCV(
    tuning_clf,
    param_grid=gs_params,
    scoring='roc_auc', 
    n_jobs=1,
    cv=5,
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)

In [ ]:
grid_search.best_estimator_
grid_search.best_params_

## Final Fit

In [ ]:
best_model = grid_search.best_estimator_

y_pred   = best_model.predict(X_test_scaled)
y_proba  = best_model.predict_proba(X_test_scaled)[:, 1]

acc      = accuracy_score(y_test, y_pred)
f1       = f1_score(y_test, y_pred)
roc_auc  = roc_auc_score(y_test, y_proba)
cm       = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {acc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC-ROC Score: {roc_auc:.4f}")
print("\nConfusion Matrix:")
print(pd.DataFrame(cm, 
                  index=['Actual Down', 'Actual Up'],
                  columns=['Predicted Down', 'Predicted Up']))

RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("ROC Curve")
plt.show()